In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import datetime

In [ ]:
# Import data from FRED
start = datetime.datetime(1953,7,1)
symbols = ['DFF', 'DTB3', 'GS1', 'GS5', 'GS10', 'USRECD', 'SP500']
market_data = pdr.fred.FredReader(symbols, freq='monthly', start=start).read()

#Group it by taking the last monthly value for each column
market_data = market_data.groupby(market_data.index.strftime('%Y-%m')).last()

In [ ]:
# Import SP500 index data to fill the FRED series (which starts from 2013)
sp500 = pd.read_csv('SPX.csv', index_col='Date')
sp500.index = pd.to_datetime(sp500.index)

# Group it by taking the last closing price for each month
sp500 = sp500['Close'].groupby(sp500.index.strftime('%Y-%m')).last().reset_index()
sp500.set_index('Date', inplace=True)

# Fill the SP500 column with the historic data
market_data['SP500'] = market_data['SP500'].combine_first(sp500['Close'])

In [ ]:
# Create the features that will be used in the models

# Labels
market_data['rec_3m'] = market_data['USRECD'].shift(-3)
market_data['rec_6m'] = market_data['USRECD'].shift(-6)
market_data['rec_12m'] = market_data['USRECD'].shift(-12)

# SP500 return
market_data['spx_6m'] = market_data['SP500']/market_data['SP500'].shift(6) - 1
market_data['spx_yoy'] = market_data['SP500']/market_data['SP500'].shift(12) - 1

# Treasury yield curve
market_data['3m_ff'] = market_data['DTB3'] - market_data['DFF']
market_data['1y_ff'] = market_data['GS1'] - market_data['DFF']
market_data['1y_3m'] = market_data['GS1'] - market_data['DTB3']
market_data['5y_1y'] = market_data['GS5'] - market_data['GS1']
market_data['10y_1y'] = market_data['GS10'] - market_data['GS1']
market_data['3m_ff_diff'] = market_data['3m_ff'].diff()
market_data['1y_ff_diff'] = market_data['1y_ff'].diff()
market_data['5y_1y_diff'] = market_data['1y_ff'].diff()
market_data['10y_1y_diff'] = market_data['10y_1y'].diff()
market_data['ff_diff'] = market_data['DFF'].diff()

In [ ]:
market_data.tail()

,DFF,DTB3,GS1,GS5,GS10,USRECD,SP500,rec_3m,rec_6m,rec_12m,...,3m_ff,1y_ff,1y_3m,5y_1y,10y_1y,3m_ff_diff,1y_ff_diff,5y_1y_diff,10y_1y_diff,ff_diff
DATE,,,,,,,,,,,,,,,,,,,,,
2022-10,3.08,4.06,4.43,4.18,3.98,0.0,3871.98,0.0,NaN,NaN,...,0.98,1.35,0.37,-0.25,-0.45,0.84,0.54,0.54,-0.08,0.00
2022-11,3.83,4.27,4.73,4.06,3.89,0.0,4080.11,0.0,NaN,NaN,...,0.44,0.90,0.46,-0.67,-0.84,-0.54,-0.45,-0.45,-0.39,0.75
2022-12,4.33,4.30,4.68,3.76,3.62,0.0,3839.50,NaN,NaN,NaN,...,-0.03,0.35,0.38,-0.92,-1.06,-0.47,-0.55,-0.55,-0.22,0.50
2023-01,4.33,4.58,4.69,3.64,3.53,0.0,4076.60,NaN,NaN,NaN,...,0.25,0.36,0.11,-1.05,-1.16,0.28,0.01,0.01,-0.10,0.00
2023-02,4.58,4.70,NaN,NaN,NaN,0.0,4079.09,NaN,NaN,NaN,...,0.12,NaN,NaN,NaN,NaN,-0.13,NaN,NaN,NaN,0.25


In [ ]:
# Rename columns and drop rows with null values
market_data.columns = ['fed_funds', '3_month', '1_year', '5_year', '10_year', 'rec', 'SP500', 'rec_3m', 'rec_6m', 'rec_12m', 'spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '3m_ff_diff', '1y_ff_diff', '5y_1y_diff', '10y_1y_diff', 'ff_diff']
market_data = market_data[market_data.index >= '1954-08']
market_data = market_data.drop(market_data.index[-1])
market_data.head()

,fed_funds,3_month,1_year,5_year,10_year,rec,SP500,rec_3m,rec_6m,rec_12m,...,3m_ff,1y_ff,1y_3m,5y_1y,10y_1y,3m_ff_diff,1y_ff_diff,5y_1y_diff,10y_1y_diff,ff_diff
DATE,,,,,,,,,,,,,,,,,,,,,
1954-08,1.44,1.05,0.88,1.90,2.36,0.0,29.83,0.0,0.0,0.0,...,-0.39,-0.56,-0.17,1.02,1.48,-0.93,-1.15,-1.15,0.02,1.19
1954-09,1.44,0.99,1.03,1.96,2.38,0.0,32.31,0.0,0.0,0.0,...,-0.45,-0.41,0.04,0.93,1.35,-0.06,0.15,0.15,-0.13,0.00
1954-10,1.13,1.00,1.17,2.02,2.43,0.0,31.68,0.0,0.0,0.0,...,-0.13,0.04,0.17,0.85,1.26,0.32,0.45,0.45,-0.09,-0.31
1954-11,1.38,1.01,1.14,2.09,2.48,0.0,34.24,0.0,0.0,0.0,...,-0.37,-0.24,0.13,0.95,1.34,-0.24,-0.28,-0.28,0.08,0.25
1954-12,1.44,1.02,1.21,2.16,2.51,0.0,35.98,0.0,0.0,0.0,...,-0.42,-0.23,0.19,0.95,1.30,-0.05,0.01,0.01,-0.04,0.06


In [ ]:
market_data.isnull().sum()

fed_funds       0
3_month         0
1_year          0
5_year          0
10_year         0
rec             0
SP500           0
rec_3m          2
rec_6m          5
rec_12m        11
spx_6m          0
spx_yoy         0
3m_ff           0
1y_ff           0
1y_3m           0
5y_1y           0
10y_1y          0
3m_ff_diff      0
1y_ff_diff      0
5y_1y_diff      0
10y_1y_diff     0
ff_diff         0
dtype: int64

In [ ]:
# Pairplot of the features to find patterns to classify recessions
import seaborn as sns
#sns.pairplot(market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '3m_ff_diff', '1y_ff_diff', '5y_1y_diff', '10y_1y_diff','ff_diff',  'rec']], hue='rec')

In [ ]:
# Correlation matrix
market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '3m_ff_diff', '1y_ff_diff', '5y_1y_diff', '10y_1y_diff', 'ff_diff', 'rec']].corr()

,spx_6m,spx_yoy,3m_ff,1y_ff,1y_3m,5y_1y,10y_1y,3m_ff_diff,1y_ff_diff,5y_1y_diff,10y_1y_diff,ff_diff,rec
spx_6m,1.000000,0.688389,0.106962,0.037573,-0.152101,0.137546,0.130906,0.004263,-0.014985,-0.014985,-0.038310,0.034290,-0.389862
spx_yoy,0.688389,1.000000,0.076297,0.009021,-0.145993,0.067641,0.062915,-0.015080,-0.019044,-0.019044,-0.089951,0.057485,-0.496731
3m_ff,0.106962,0.076297,1.000000,0.890708,-0.284302,0.297000,0.309227,0.457358,0.430653,0.430653,-0.036164,-0.373406,-0.222391
1y_ff,0.037573,0.009021,0.890708,1.000000,0.182588,0.359521,0.311447,0.433694,0.468609,0.468609,0.025809,-0.437433,-0.121880
1y_3m,-0.152101,-0.145993,-0.284302,0.182588,1.000000,0.115884,-0.011953,-0.074514,0.056885,0.056885,0.132651,-0.114951,0.223948
5y_1y,0.137546,0.067641,0.297000,0.359521,0.115884,1.000000,0.967523,0.021386,-0.005696,-0.005696,0.138521,-0.057534,-0.054529
10y_1y,0.130906,0.062915,0.309227,0.311447,-0.011953,0.967523,1.000000,0.024743,-0.004603,-0.004603,0.128051,-0.057727,-0.070369
3m_ff_diff,0.004263,-0.015080,0.457358,0.433694,-0.074514,0.021386,0.024743,1.000000,0.933548,0.933548,0.075585,-0.891452,0.024216
1y_ff_diff,-0.014985,-0.019044,0.430653,0.468609,0.056885,-0.005696,-0.004603,0.933548,1.000000,1.000000,0.013619,-0.909384,0.026387
5y_1y_diff,-0.014985,-0.019044,0.430653,0.468609,0.056885,-0.005696,-0.004603,0.933548,1.000000,1.000000,0.013619,-0.909384,0.026387


In [ ]:
# Select inputs for the first model (predict recessions in real time)
y = market_data['rec'].dropna()
X = market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '10y_1y', '10y_1y_diff', 'ff_diff']].loc[y.index]

In [ ]:
# Classes are highly imbalanced
y.value_counts()

0.0    719
1.0    103
Name: rec, dtype: int64

In [ ]:
# Set random state variable for future uses
random_state=3

In [ ]:
# Split the data into training and testing sets, and transform training data to address class imbalance using ADASYN
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state, stratify=y)
X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)

In [ ]:
# Create the model using a GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=random_state, n_estimators=300, subsample=0.4, min_samples_leaf=8)

# Use a forward floating feature selector, using cross validation and the F1 score to choose between models 
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs1 = SFS(model, k_features = 6, forward=True, floating=True, scoring='f1', cv=5)
sfs1.fit(X_resampled, y_resampled)

# Visualize the different combination of features
sfs1.subsets_

{1: {'feature_idx': (0,),
  'cv_scores': array([0.83193277, 0.77253219, 0.85964912, 0.85964912, 0.82407407]),
  'avg_score': 0.829567456327711,
  'feature_names': ('0',)},
 2: {'feature_idx': (0, 5),
  'cv_scores': array([0.87336245, 0.86192469, 0.87931034, 0.90987124, 0.87850467]),
  'avg_score': 0.8805946787934582,
  'feature_names': ('0', '5')},
 3: {'feature_idx': (0, 2, 5),
  'cv_scores': array([0.86995516, 0.88235294, 0.91555556, 0.92173913, 0.87155963]),
  'avg_score': 0.892232483429001,
  'feature_names': ('0', '2', '5')},
 4: {'feature_idx': (0, 1, 2, 5),
  'cv_scores': array([0.90350877, 0.88888889, 0.93449782, 0.94420601, 0.90825688]),
  'avg_score': 0.9158716733460471,
  'feature_names': ('0', '1', '2', '5')},
 5: {'feature_idx': (0, 1, 2, 3, 5),
  'cv_scores': array([0.91304348, 0.90909091, 0.96069869, 0.94827586, 0.91818182]),
  'avg_score': 0.9298581515117788,
  'feature_names': ('0', '1', '2', '3', '5')},
 6: {'feature_idx': (0, 1, 2, 3, 4, 5),
  'cv_scores': array([0.9

In [ ]:
# Transform the features
X_train_sfs = sfs1.transform(X_resampled)
X_test_sfs = sfs1.transform(X_test)

# Fit the model using the chosen features
model.fit(X_train_sfs, y_resampled)

# Print the confusion matrix using the testing data
confusion_matrix(y_test, model.predict(X_test_sfs))

array([[135,   9],
       [  1,  20]])

In [ ]:
# To evaluate the model, use the classification report for imbalanced data
from imblearn.metrics import classification_report_imbalanced
classification_report_imbalanced(y_test, model.predict(X_test_sfs), output_dict=True)

{0.0: {'pre': 0.9926470588235294,
  'rec': 0.9375,
  'spe': 0.9523809523809523,
  'f1': 0.9642857142857143,
  'geo': 0.944911182523068,
  'iba': 0.8915284863945577,
  'sup': 144},
 1.0: {'pre': 0.6896551724137931,
  'rec': 0.9523809523809523,
  'spe': 0.9375,
  'f1': 0.7999999999999999,
  'geo': 0.944911182523068,
  'iba': 0.894185799319728,
  'sup': 21},
 'avg_pre': 0.9540844550986538,
 'avg_rec': 0.9393939393939394,
 'avg_spe': 0.950487012987013,
 'avg_f1': 0.9433766233766233,
 'avg_geo': 0.9449111825230679,
 'avg_iba': 0.8918666898577612,
 'total_support': 165}

In [ ]:
# Pairplot of the features to find patterns to classify recessions with a 3 month anticipation
#sns.pairplot(market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '3m_ff_diff', '1y_ff_diff', '5y_1y_diff', '10y_1y_diff', 'rec_3m']], hue='rec_3m')

In [ ]:
# Correlation matrix
market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '3m_ff_diff', '1y_ff_diff', '5y_1y_diff', '10y_1y_diff', 'ff_diff', 'rec_3m']].corr()

,spx_6m,spx_yoy,3m_ff,1y_ff,1y_3m,5y_1y,10y_1y,3m_ff_diff,1y_ff_diff,5y_1y_diff,10y_1y_diff,ff_diff,rec_3m
spx_6m,1.000000,0.688389,0.106962,0.037573,-0.152101,0.137546,0.130906,0.004263,-0.014985,-0.014985,-0.038310,0.034290,-0.437593
spx_yoy,0.688389,1.000000,0.076297,0.009021,-0.145993,0.067641,0.062915,-0.015080,-0.019044,-0.019044,-0.089951,0.057485,-0.444851
3m_ff,0.106962,0.076297,1.000000,0.890708,-0.284302,0.297000,0.309227,0.457358,0.430653,0.430653,-0.036164,-0.373406,-0.286092
1y_ff,0.037573,0.009021,0.890708,1.000000,0.182588,0.359521,0.311447,0.433694,0.468609,0.468609,0.025809,-0.437433,-0.213339
1y_3m,-0.152101,-0.145993,-0.284302,0.182588,1.000000,0.115884,-0.011953,-0.074514,0.056885,0.056885,0.132651,-0.114951,0.168690
5y_1y,0.137546,0.067641,0.297000,0.359521,0.115884,1.000000,0.967523,0.021386,-0.005696,-0.005696,0.138521,-0.057534,-0.245471
10y_1y,0.130906,0.062915,0.309227,0.311447,-0.011953,0.967523,1.000000,0.024743,-0.004603,-0.004603,0.128051,-0.057727,-0.245617
3m_ff_diff,0.004263,-0.015080,0.457358,0.433694,-0.074514,0.021386,0.024743,1.000000,0.933548,0.933548,0.075585,-0.891452,0.043587
1y_ff_diff,-0.014985,-0.019044,0.430653,0.468609,0.056885,-0.005696,-0.004603,0.933548,1.000000,1.000000,0.013619,-0.909384,0.072416
5y_1y_diff,-0.014985,-0.019044,0.430653,0.468609,0.056885,-0.005696,-0.004603,0.933548,1.000000,1.000000,0.013619,-0.909384,0.072416


In [ ]:
# Split the data and transform it to address class imbalance
y_2 = market_data['rec_3m'].dropna()
X_2 = market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '10y_1y_diff', 'ff_diff']].loc[y_2.index]
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2, test_size=0.2, random_state=random_state, stratify=y_2)
X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)

In [ ]:
# Create the model using a GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
model2 = GradientBoostingClassifier(random_state=random_state, n_estimators=300, subsample=0.4, min_samples_leaf=8)

# Use a forward floating feature selector, using cross validation and the F1 score to choose between models 
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs2 = SFS(model, k_features = 6, forward=True, floating=True, scoring='f1', cv=5)
sfs2.fit(X_resampled, y_resampled)

# Visualize the different combination of features
sfs2.subsets_



STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

{1: {'feature_idx': (1,),
  'cv_scores': array([0.82786885, 0.81781377, 0.81102362, 0.75652174, 0.83950617]),
  'avg_score': 0.8105468303316774},
 2: {'feature_idx': (1, 5),
  'cv_scores': array([0.89344262, 0.85714286, 0.88979592, 0.87603306, 0.91764706]),
  'avg_score': 0.8868123030271586}}

In [ ]:
# Transform the features
X_train_sfs = sfs2.transform(X_resampled)
X_test_sfs = sfs2.transform(X_test)

# Fit the model using the chosen features
model2.fit(X_train_sfs, y_resampled)

# Print the confusion matrix using the testing data
confusion_matrix(y_test, model2.predict(X_test_sfs))

AttributeError: ignored

In [ ]:
# To evaluate the model, use the classification report for imbalanced data
classification_report_imbalanced(y_test, model2.predict(X_test_sfs), output_dict=True)

In [ ]:
# Pairplot of the features to find patterns to classify recessions with a 6 month anticipation
#sns.pairplot(market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '3m_ff_diff', '1y_ff_diff', '5y_1y_diff', '10y_1y_diff', 'rec_6m']], hue='rec_6m')

In [ ]:
# Correlation matrix
market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '3m_ff_diff', '1y_ff_diff', '5y_1y_diff', '10y_1y_diff', 'ff_diff', 'rec_6m']].corr()

In [ ]:
# Select inputs for the third model, split the data and transform it
y_3 = market_data['rec_6m'].dropna()
X_3 = market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '5y_1y', '10y_1y', '10y_1y_diff']].loc[y_3.index]
X_train, X_test, y_train, y_test = train_test_split(X_3, y_3, test_size=0.2, random_state=random_state, stratify=y_3)
X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)

In [ ]:
# Create the model using a GradientBoostingClassifier
model3 = GradientBoostingClassifier(random_state=random_state, n_estimators=300, subsample=0.4, min_samples_leaf=8)

# Use a forward floating feature selector, using cross validation and the F1 score to choose between models 
sfs3 = SFS(model, k_features = 6, forward=True, floating=True, scoring='f1', cv=5)
sfs3.fit(X_resampled, y_resampled)

# Visualize the different combination of features
sfs3.subsets_

In [ ]:
# Transform the features
X_train_sfs = sfs3.transform(X_resampled)
X_test_sfs = sfs3.transform(X_test)

# Fit the model using the chosen features
model3.fit(X_train_sfs, y_resampled)

# Print the confusion matrix using the testing data
confusion_matrix(y_test, model3.predict(X_test_sfs))

In [ ]:
# To evaluate the model, use the classification report for imbalanced data
classification_report_imbalanced(y_test, model3.predict(X_test_sfs), output_dict=True)

In [ ]:
# Pairplot of the features to find patterns to classify recessions with a 12 month anticipation
#sns.pairplot(market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '3m_ff_diff', '1y_ff_diff', '5y_1y_diff', '10y_1y_diff', 'rec_12m']], hue='rec_12m')

In [ ]:
# Correlation matrix
market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m', '5y_1y', '10y_1y', '3m_ff_diff', '1y_ff_diff', '5y_1y_diff', '10y_1y_diff', 'ff_diff', 'rec_12m']].corr()

In [ ]:
# Select inputs for the fourth model, split the data and transform it
y_4 = market_data['rec_12m'].dropna()
X_4 =  market_data[['spx_6m', 'spx_yoy', '3m_ff', '1y_ff', '1y_3m','5y_1y', '10y_1y']].loc[y_4.index]
X_train, X_test, y_train, y_test = train_test_split(X_4, y_4, test_size=0.2, random_state=random_state, stratify=y_4)
X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)

In [ ]:
# Create the model using a GradientBoostingClassifier
model4 = GradientBoostingClassifier(random_state=random_state, n_estimators=300, subsample=0.4, min_samples_leaf=8)

# Use a forward floating feature selector, using cross validation and the F1 score to choose between models 
sfs4 = SFS(model, k_features = 5, forward=True, floating=True, scoring='f1', cv=5)
sfs4.fit(X_resampled, y_resampled)

# Visualize the different combination of features
sfs4.subsets_

In [ ]:
# Transform the features
X_train_sfs = sfs4.transform(X_resampled)
X_test_sfs = sfs4.transform(X_test)

# Fit the model using the chosen features
model4.fit(X_train_sfs, y_resampled)

# Print the confusion matrix using the testing data
confusion_matrix(y_test, model4.predict(X_test_sfs))

In [ ]:
# To evaluate the model, use the classification report for imbalanced data
classification_report_imbalanced(y_test, model4.predict(X_test_sfs), output_dict=True)

In [ ]:
# Fit the first model on the entire transformed data
X = sfs1.transform(X)
X_resampled, y_resampled = ADASYN().fit_resample(X, y)
model.fit(X_resampled, y_resampled)

In [ ]:
# Fit the second model on the entire transformed data
X_2 = sfs2.transform(X_2)
X_resampled, y_resampled = ADASYN().fit_resample(X_2, y_2)
model2.fit(X_resampled, y_resampled)

In [ ]:
# Fit the third model on the entire transformed data
X_3 = sfs3.transform(X_3)
X_resampled, y_resampled = ADASYN().fit_resample(X_3, y_3)
model3.fit(X_resampled, y_resampled)

In [ ]:
# Fit the fourth model on the entire transformed data
X_4 = sfs4.transform(X_4)
X_resampled, y_resampled = ADASYN().fit_resample(X_4, y_4)
model4.fit(X_resampled, y_resampled)

In [ ]:
# Create columns using the predictions
market_data['rec_prob'] = model.predict_proba(market_data[list(sfs1.subsets_[6]['feature_names'])])[:, 1]
market_data['rec_3m_prob'] = model2.predict_proba(market_data[list(sfs2.subsets_[6]['feature_names'])])[:, 1]
market_data['rec_6m_prob'] = model3.predict_proba(market_data[list(sfs3.subsets_[6]['feature_names'])])[:, 1]
market_data['rec_12m_prob'] = model4.predict_proba(market_data[list(sfs4.subsets_[5]['feature_names'])])[:, 1]

In [ ]:
# Plot the data using gray shading for recessionary periods
import matplotlib.pyplot as plt
market_data.index = pd.to_datetime(market_data.index)
fig, ax = plt.subplots(figsize=(20,10))
plt.plot(market_data.index, market_data.rec_prob, color='blue')
plt.plot(market_data.index, market_data.rec_3m_prob, color='green')
plt.plot(market_data.index, market_data.rec_6m_prob, color='red')
plt.plot(market_data.index, market_data.rec_12m_prob, color='purple')
ax.fill_between(market_data.index, market_data.rec, where=(market_data.rec==1), color='gray', alpha=0.5)
plt.legend(['Probability of recession', 'Probability of recession in 3 months', 'Probability of recession in 6 months', 'Probability of recession in 12 months', 'Recession'])
plt.show()
plt.close()